In [1]:
from deepface import DeepFace

### New Virsion

In [4]:
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQL@2025",
    database="smart_attendance"
)
cursor = conn.cursor()

marked_students = set()

# ==============================
# Function: Get current lecture
# ==============================
def get_current_lecture():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT lecture_id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    cursor.execute(query, (today, current_time, current_time))
    return cursor.fetchone()

# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    lecture = get_current_lecture()

    if not lecture:
        cv2.putText(frame, "No Active Lecture", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    lecture_id, subject, course, year, classroom = lecture

    try:
        res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

        if len(res[0]) > 0:
            name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

            # Get roll number
            cursor.execute("SELECT student_roll_no FROM students WHERE name = %s", (name,))
            student_row = cursor.fetchone()

            if student_row:
                student_roll_no = student_row[0]

                # Insert a scan entry (for each detection)
                cursor.execute("""
                    INSERT INTO lecture_scans (lecture_id, student_roll_no, scan_no, appeared)
                    VALUES (%s, %s, %s, %s)
                """, (lecture_id, student_roll_no, 1, 1))
                conn.commit()

                # Count total scans for this student in this lecture
                cursor.execute("""
                    SELECT COUNT(*) FROM lecture_scans
                    WHERE lecture_id = %s AND student_roll_no = %s
                """, (lecture_id, student_roll_no))
                scan_count = cursor.fetchone()[0]

                # If appeared at least 3 times, mark attendance
                if scan_count >= 3 and (student_roll_no, lecture_id) not in marked_students:
                    today = datetime.now().date()
                    cursor.execute("""
                        INSERT INTO attendance (lecture_id, student_roll_no, name, status, date)
                        VALUES (%s, %s, %s, %s, %s)
                    """, (lecture_id, student_roll_no, name, 'Present', today))
                    conn.commit()

                    marked_students.add((student_roll_no, lecture_id))
                    print(f"✅ Final Attendance Marked for {name} ({student_roll_no}) in Lecture {lecture_id}")

            # Optional: Draw bounding box if DeepFace returned coordinates
            if 'source_x' in res[0]:
                x = int(res[0]['source_x'][0])
                y = int(res[0]['source_y'][0])
                w = int(res[0]['source_w'][0])
                h = int(res[0]['source_h'][0])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, name, (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    except Exception as e:
        print("Error:", e)

    cv2.imshow("Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
cursor.close()
conn.close()


25-11-08 14:49:46 - Searching [[[ 29  43  60]
  [ 25  39  56]
  [ 24  39  56]
  ...
  [171 181 187]
  [172 183 185]
  [172 184 183]]

 [[ 25  39  56]
  [ 25  39  56]
  [ 25  41  57]
  ...
  [171 183 182]
  [172 184 182]
  [172 184 183]]

 [[ 23  37  54]
  [ 25  39  56]
  [ 25  41  57]
  ...
  [171 182 183]
  [171 182 183]
  [171 182 183]]

 ...

 [[145 156 160]
  [147 157 161]
  [148 158 162]
  ...
  [ 97 120 129]
  [120 147 155]
  [150 180 186]]

 [[144 155 158]
  [147 157 161]
  [148 158 162]
  ...
  [ 99 124 135]
  [100 126 135]
  [138 163 170]]

 [[144 155 158]
  [145 156 160]
  [147 157 161]
  ...
  [102 126 138]
  [ 95 122 132]
  [118 148 157]]] in 6 length datastore
25-11-08 14:49:47 - find function duration 0.4324512481689453 seconds
✅ Final Attendance Marked for Rameshwar Kale (2207025) in Lecture 3
25-11-08 14:49:47 - Searching [[[ 17  21  21]
  [ 16  20  20]
  [ 14  20  17]
  ...
  [138 146 143]
  [139 147 144]
  [139 147 144]]

 [[ 17  21  21]
  [ 16  20  20]
  [ 14  19  19

### Time Intervel update

In [3]:
import cv2
import time
import mysql.connector
from datetime import datetime, timedelta
from deepface import DeepFace

# ==============================
# CONFIG
# ==============================
db_path = r"D:\Smart_Attendance\database"
SCAN_INTERVAL = 15 * 60   # 15 minutes in seconds
TOTAL_SCANS = 4

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="MySQL@2025",
    database="smart_attendance"
)
cursor = conn.cursor()

# Track already marked students
marked_students = set()


# ==============================
# Function: Get current lecture
# ==============================
def get_current_lecture():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT lecture_id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    cursor.execute(query, (today, current_time, current_time))
    return cursor.fetchone()


# ==============================
# CAMERA & ATTENDANCE
# ==============================
cap = cv2.VideoCapture(0)

current_scan_no = 1
last_scan_time = datetime.now() - timedelta(seconds=SCAN_INTERVAL)  # trigger immediate first scan

print("\n📸 Smart Attendance System Started\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    lecture = get_current_lecture()

    if not lecture:
        cv2.putText(frame, "No Active Lecture", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow("Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    lecture_id, subject, course, year, classroom = lecture

    # Check if it's time for the next scan
    now = datetime.now()
    if (now - last_scan_time).total_seconds() >= SCAN_INTERVAL:
        print(f"\n🔍 Starting Scan {current_scan_no} at {now.strftime('%H:%M:%S')}")
        last_scan_time = now

        try:
            res = DeepFace.find(frame, db_path=db_path, enforce_detection=False, model_name='VGG-Face')

            if len(res[0]) > 0:
                name = res[0]['identity'].iloc[0].replace('/', '\\').split('\\')[-2]

                # Get roll number from students table
                cursor.execute("SELECT student_roll_no FROM students WHERE name = %s", (name,))
                student_row = cursor.fetchone()

                if student_row:
                    student_roll_no = student_row[0]

                    # Insert scan record
                    cursor.execute("""
                        INSERT INTO lecture_scans (lecture_id, student_roll_no, scan_no, appeared)
                        VALUES (%s, %s, %s, %s)
                    """, (lecture_id, student_roll_no, current_scan_no, 1))
                    conn.commit()

                    print(f"✅ Detected {name} (Roll: {student_roll_no}) in Scan {current_scan_no}")

            else:
                print("⚠️ No student detected in this scan.")

        except Exception as e:
            print("Error during scan:", e)

        current_scan_no += 1
        if current_scan_no > TOTAL_SCANS:
            print("\n🧮 All scans completed. Calculating final attendance...\n")
            break

    # Show live camera
    cv2.imshow("Attendance", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("\n🛑 Exiting manually...\n")
        break


# ==============================
# FINAL ATTENDANCE CALCULATION
# ==============================
if lecture:
    lecture_id, subject, course, year, classroom = lecture
    today = datetime.now().date()

    cursor.execute("""
        SELECT student_roll_no, COUNT(*) AS total_scans
        FROM lecture_scans
        WHERE lecture_id = %s
        GROUP BY student_roll_no
    """, (lecture_id,))

    results = cursor.fetchall()

    for student_roll_no, total_scans in results:
        status = 'Present' if total_scans >= 3 else 'Absent'

        # Fetch student name
        cursor.execute("SELECT name FROM students WHERE student_roll_no = %s", (student_roll_no,))
        name = cursor.fetchone()[0]

        cursor.execute("""
            INSERT INTO attendance (lecture_id, student_roll_no, name, status, date)
            VALUES (%s, %s, %s, %s, %s)
        """, (lecture_id, student_roll_no, name, status, today))
        conn.commit()

        print(f"📋 Final Attendance: {name} ({student_roll_no}) → {status}")

    print("\n✅ Final attendance records successfully inserted.\n")


# ==============================
# CLEANUP
# ==============================
cap.release()
cv2.destroyAllWindows()
cursor.close()
conn.close()

print("🔒 System closed.")



📸 Smart Attendance System Started

🔒 System closed.


## Web Interface

In [1]:
from flask import Flask, jsonify, Response
import cv2
import mysql.connector
from datetime import datetime
from deepface import DeepFace
import os 

# 1. INITIALIZE APP
app = Flask(__name__)

# 2. CONFIGURATION / GLOBAL VARIABLES
db_path = r"D:\Smart_Attendance\database"
marked_students = set()

# Initialize connection and cursor to None globally (best practice for routes)
conn = None
cursor = None

# 3. HELPER FUNCTION (Needs the global 'cursor' to be set)
def get_current_lecture():
    # ... (Keep this function as is from the previous correction)
    if not cursor:
        # If cursor is not set, return None gracefully
        return None
        
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    today = now.date()

    query = """
        SELECT id, subject, course, year, classroom
        FROM lectures
        WHERE date = %s
        AND start_time <= %s
        AND end_time >= %s
        LIMIT 1
    """
    try:
        cursor.execute(query, (today, current_time, current_time))
        lecture = cursor.fetchone()
        return lecture
    except mysql.connector.Error as e:
        print(f"❌ Database error in get_current_lecture: {e}")
        return None

# 4. API: Get Current Attendance
@app.route("/api/current-attendance")
def current_attendance():
    # ... (Keep this function as is from the previous correction)
    if not cursor:
        return jsonify({"status": "Error", "message": "Database not connected"}), 500
        
    lecture = get_current_lecture()
    # ... (Rest of the logic)
    
    # [Insert the rest of the current_attendance function here]
    if not lecture:
        return jsonify({"status": "No active lecture"})

    lecture_id, subject, course, year, classroom = lecture
    try:
        cursor.execute("""
            SELECT s.name, a.status, a.entry_time
            FROM attendance a
            JOIN students s ON a.student_id = s.student_id
            WHERE a.lecture_id = %s
        """, (lecture_id,))
        records = cursor.fetchall()
    except mysql.connector.Error as e:
        return jsonify({"status": "Error", "message": "Could not fetch attendance"}), 500

    return jsonify({
        "lecture": {
            "id": lecture_id,
            "subject": subject,
            "course": course,
            "year": year,
            "classroom": classroom
        },
        "students": [{"name": r[0], "status": r[1], "entry_time": str(r[2])} for r in records]
    })


# 5. API: Video Feed (Streaming) - Generator Function
def gen_frames():
    # ... (Keep this function as is from the previous correction, it is essential)
    # 🌟 CRUCIAL FIX: Initialize VideoCapture here
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Error: Could not open video stream.")
        # If camera fails, yield a simple error frame or break
        return

    while True:
        # [Insert the rest of the gen_frames function here]
        success, frame = cap.read()
        if not success:
            print("Video capture failed.")
            break
            
        frame = cv2.flip(frame, 1)
        lecture = get_current_lecture()
        
        # Attendance Logic
        if lecture and cursor and conn:
            lecture_id, _, _, _, _ = lecture
            try:
                res = DeepFace.find(
                    frame, 
                    db_path=db_path, 
                    enforce_detection=False, 
                    model_name='VGG-Face',
                )
                
                if res and len(res[0]) > 0 and 'identity' in res[0] and len(res[0]['identity']) > 0:
                    identity_path = res[0]['identity'].iloc[0]
                    student_dir = os.path.basename(os.path.dirname(identity_path))
                    name = student_dir
                    
                    cursor.execute("SELECT student_id FROM students WHERE name = %s", (name,))
                    student_row = cursor.fetchone()

                    if student_row:
                        student_id = student_row[0]
                        if (student_id, lecture_id) not in marked_students:
                            entry_time = datetime.now().strftime("%H:%M:%S")

                            cursor.execute("""
                                INSERT INTO lecture_scans (lecture_id, student_id, scan_no, appeared)
                                VALUES (%s, %s, %s, %s)
                                ON DUPLICATE KEY UPDATE appeared = 1
                            """, (lecture_id, student_id, 1, 1))
                            
                            cursor.execute("""
                                INSERT INTO attendance (lecture_id, student_id, status, entry_time)
                                VALUES (%s, %s, %s, %s)
                                ON DUPLICATE KEY UPDATE status = VALUES(status), entry_time = VALUES(entry_time)
                            """, (lecture_id, student_id, "Present", entry_time))

                            conn.commit()
                            marked_students.add((student_id, lecture_id))
                            print(f"✅ {name} marked present in Lecture {lecture_id}")

            except Exception as e:
                # print(f"❌ Processing error: {e}") # Debugging
                pass

        # Encode frame to stream
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
               
    cap.release()

@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')


# 6. MAIN EXECUTION BLOCK (The critical fix is here)
if __name__ == "__main__":
    
    # 🌟 CRITICAL FIX: Establish DB connection just before run
    # and assign it to the global variables.
    try:
        # Re-assigning to global variables used by the routes/helpers
        globals()['conn'] = mysql.connector.connect(
            host="localhost",
            user="root",
            password="MySQL@2025",
            database="smart_attendance"
        )
        globals()['cursor'] = conn.cursor()
        print("✅ MySQL Database connected successfully (in main thread).")
    except mysql.connector.Error as e:
        print(f"❌ MySQL error: {e}")
        # If DB connection fails, the global conn/cursor remain None

    try:
        # 🌟 CRITICAL FIX: Use use_reloader=False to stop the second process from running.
        # This will eliminate the SystemExit: 1 caused by DeepFace/OpenCV resource conflicts.
        app.run(debug=True, use_reloader=False) 
    except Exception as e:
        print(f"❌ Flask crashed: {e}")

✅ MySQL Database connected successfully (in main thread).
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Oct/2025 11:31:10] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [08/Oct/2025 11:31:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Oct/2025 11:31:27] "GET /video_feed HTTP/1.1" 200 -


❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'
❌ Database error in get_current_lecture: 1054 (42S22): Unknown column 'id' in 'field list'

25-10-08 07:44:41 - Searching [[[115 128 132]
  [115 128 132]
  [116 128 133]
  ...
  [115 131 141]
  [116 131 140]
  [117 131 139]]

 [[113 128 133]
  [113 128 133]
  [113 128 133]
  ...
  [116 132 140]
  [117 132 140]
  [118 132 140]]

 [[113 129 132]
  [113 129 132]
  [113 129 132]
  ...
  [117 131 137]
  [118 132 139]
  [118 132 140]]

 ...

 [[244 255 255]
  [244 255 255]
  [244 255 255]
  ...
  [118 127 129]
  [118 127 129]
  [117 126 128]]

 [[244 255 255]
  [244 255 255]
  [244 255 255]
  ...
  [118 127 129]
  [118 127 129]
  [117 126 128]]

 [[244 255 255]
  [244 255 255]
  [244 255 255]
  ...
  [120 128 130]
  [118 127 129]
  [117 126 128]]] in 5 length datastore
25-10-08 07:44:42 - find function duration 0.7182557582855225 seconds
Attendance marked: Rameshwar Kale (Unknown) at 07:44:42
25-10-08 07:44:42 - Searching [[[115 125 131]
  [115 125 131]
  [115 125 131]
  ...
  [119 132 135]
  [119 132 134]
  [119 132 133]]

 [[115 125 131]
  [115 125 131]
  [115 125 131]
  ...
  [1

KeyboardInterrupt: 